# Regression LSTM with best parameters
    find the best prediction window to apply w/ lr = 0.0001

# Import libraries

In [21]:
!pip install pandas
!pip install sklearn
!pip install matplotlib

import pandas as pd;
import numpy as np;
import sklearn
import matplotlib.pyplot as plt
import time

date =  [2018010000, 
         2018030000, 
         2018050000,
         2018070000,
         2018090000,
         2018110000]

# parameters
steps = 336
n_hidden = 2
units = 100
batch_size = 96
epochs = 180
features_num = 14

# lists to append results
mae_gen = []
mae_nor = []
mae_spi = []
rmse_gen = []
rmse_nor = []
rmse_spi = []
y_pred_list = []
time_count = []

# Import keras libraries, packages and data:

In [22]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import LeakyReLU
from keras import initializers
from keras import optimizers

# import data
data_full = pd.read_csv('Data_set_1_smaller_(1).csv', index_col = 0)

# Create loop for different dates:

In [ ]:
# function to split data into correct shape for RNN
def split_data(X, y, steps):
    X_, y_ = list(), list()
    for i in range(steps, len(y)):
        X_.append(X[i - steps : i, :])
        y_.append(y[i]) 
    return np.array(X_), np.array(y_)

# function to cut data set so it can be divisible by the batch_size
def cut_data(data, batch_size):
     # see if it is divisivel
    condition = data.shape[0] % batch_size
    if condition == 0:
        return data
    else:
        return data[: -condition]

def regressor_tunning(kernel_initializer = 'he_uniform',
                      bias_initializer = initializers.Ones()):
    model = Sequential()
    if n_hidden == 0:
        model.add(LSTM(units = units,                    
                       input_shape = (steps, features_num), 
                       kernel_initializer = kernel_initializer,
                       bias_initializer = bias_initializer))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Dropout(0.2))
    else:
        model.add(LSTM(units = units,                    
                       input_shape = (steps, features_num), 
                       return_sequences = True,
                       kernel_initializer = kernel_initializer,
                       bias_initializer = bias_initializer))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Dropout(0.2))
        model.add(LSTM(units = units, 
                       input_shape = (steps, features_num), 
                       kernel_initializer = kernel_initializer,
                       bias_initializer = bias_initializer))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear'))
    optimizer = optimizers.RMSprop()
    model.compile(loss = 'mse', metrics = ['mse', 'mae'], optimizer = optimizer)
    return model
  
# LOOP STARTS
for i in date:
    start_time = time.time()
    # data
    data = data_full.loc[data_full.index > i, :]

    # reset index
    data.reset_index(inplace = True)
    data.drop('index', axis = 1, inplace = True)

    # fill nan values in the whole data set
    data.fillna(data.mean(), inplace = True)

    from sklearn.model_selection import train_test_split

    # divide data into train and test 
    data_train, data_test = train_test_split(
             data, test_size = 0.15, shuffle=False)  
    
    from sklearn.preprocessing import MinMaxScaler

    # data scaling  (including offer (y))
    sc_X = MinMaxScaler()
    data_train = sc_X.fit_transform(data_train)
    data_test = sc_X.transform(data_test)
    
    # divide features and labels
    X_train = data_train[:, 0:14] 
    y_train = data_train[:, -1]
    X_test = data_test[:, 0:14] 
    y_test = data_test[:, -1] 

    # divide data into train and test 
    X_train, X_val, y_train, y_val = train_test_split(
             X_train, y_train, test_size = 0.10, shuffle=False)

    # put data into correct shape
    X_train, y_train = split_data(X_train, y_train, steps)
    X_test, y_test = split_data(X_test, y_test, steps)
    X_val, y_val = split_data(X_val, y_val, steps)

    X_train = cut_data(X_train, batch_size)
    y_train = cut_data(y_train, batch_size)
    X_test = cut_data(X_test, batch_size)
    y_test = cut_data(y_test, batch_size)
    X_val = cut_data(X_val, batch_size)
    y_val = cut_data(y_val, batch_size)

    model = regressor_tunning()
    
    # fitting the LSTM to the training set
    history = model.fit(X_train,
                        y_train, 
                        batch_size = batch_size, 
                        epochs = epochs,
                        shuffle = False)
                        #validation_data = (X_val, y_val))
    
    model.reset_states()
    
    # make new predicitons with test set
    y_pred = model.predict(X_test, batch_size = batch_size)
    
    # prices col = 15 (inverso should not be used as scalling was made with the whole data set)
    y_pred = (y_pred * sc_X.data_range_[14]) + (sc_X.data_min_[14])
    y_test = (y_test * sc_X.data_range_[14]) + (sc_X.data_min_[14])

    # smal adjustment
    y_test = pd.Series(y_test)
    y_test.replace(0, 0.0001,inplace = True)
    
    y_pred_list.append(y_pred)
    
    from sklearn.metrics import mean_squared_error as mse
    from sklearn.metrics import mean_absolute_error as mae

    rmse_error = mse(y_test, y_pred, squared = False)
    mae_error = mae(y_test, y_pred)
    
    rmse_gen.append(rmse_error)
    mae_gen.append(mae_error)
    
    # =============================================================================
    # Metrics evaluation on spike regions
    # =============================================================================
    
    # Need to process data with spike occurences the same way as features
    data = pd.read_csv('Spike_binary_1std.csv', index_col = 0)

    # set predictive window according with tuning best results
    data = data.loc[data.index > i, :]

    # make sure shaded area will correspond to values outputed by LSTM
    data.reset_index(drop = True, inplace = True)

    # fill_nan is already made - so lets split data into test and train
    from sklearn.model_selection import train_test_split

    # divide data into train and test 
    shade_train, shade_test = train_test_split(
             data, test_size = 0.15, shuffle = False)

    # reset index of testing data
    shade_test.reset_index(drop = True, inplace = True)

    # function to split data into correct shape for RNN
    def split_data_shade(shade_test, steps):
        y_spike_occ = list()
        upper_lim = list()
        lower_lim = list()
        for i in range(steps, len(shade_test.index)):
            y_spike_occ.append(shade_test['spike_occurance'][i])
            upper_lim.append(shade_test['spike_upperlim'][i])
            lower_lim.append(shade_test['spike_lowerlim'][i])
        return np.array(y_spike_occ), np.array(upper_lim), np.array(lower_lim)

    # function to cut data set so it can be divisible by the batch_size
    def cut_data_shade(data, batch_size):
         # see if it is divisivel
        condition = data.shape[0] % batch_size
        if condition == 0:
            return data
        else:
            return data[: -condition]
    
    # shape y_spike_occ for the right size to compare results in normal and spike regions
    y_spike_occ, spike_upperlim, spike_lowerlim = split_data_shade(shade_test, steps)
    y_spike_occ = cut_data_shade(y_spike_occ, batch_size)

    # continue
    
    # select y_pred and y_test only for regions with spikes
    y_test_spike = (y_test.T * y_spike_occ).T
    y_pred_spike = (y_pred.T * y_spike_occ).T
    y_test_spike = y_test_spike[y_test_spike != 0]
    y_pred_spike = y_pred_spike[y_pred_spike != 0]
    
    # calculate metric
    rmse_spike = mse(y_test_spike, y_pred_spike, squared = False)
    mae_spike = mae(y_test_spike, y_pred_spike)
    
    rmse_spi.append(rmse_spike)
    mae_spi.append(mae_spike)
    
    # =============================================================================
    # Metric evaluation on normal regions
    # =============================================================================
    
    # inverse y_spike_occ so the only normal occurences are chosen
    y_normal_occ = (y_spike_occ - 1) * (-1)
    
    # sanity check
    y_normal_occ.sum() + y_spike_occ.sum() # gives the correct total 
    
    # select y_pred and y_test only for normal regions
    y_test_normal = (y_test.T * y_normal_occ).T
    y_pred_normal = (y_pred.T * y_normal_occ).T
    y_test_normal = y_test_normal[y_test_normal != 0.00]
    y_pred_normal = y_pred_normal[y_pred_normal != 0.00]
    
    # calculate metric
    rmse_normal = mse(y_test_normal, y_pred_normal, squared = False)
    mae_normal = mae(y_test_normal, y_pred_normal)
    
    rmse_nor.append(rmse_normal)
    mae_nor.append(mae_normal)
    
    elapsed_time = time.time() - start_time

    time_count.append(elapsed_time)

C:\Users\maria\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\maria\anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


Epoch 1/180
136/136 [==============================] - 166s 1s/step - loss: 0.1318 - mse: 0.1318 - mae: 0.2607
Epoch 2/180
136/136 [==============================] - 167s 1s/step - loss: 0.0096 - mse: 0.0096 - mae: 0.0720
Epoch 3/180
136/136 [==============================] - 169s 1s/step - loss: 0.0023 - mse: 0.0023 - mae: 0.0311
Epoch 4/180
136/136 [==============================] - 165s 1s/step - loss: 0.0015 - mse: 0.0015 - mae: 0.0231
Epoch 5/180
136/136 [==============================] - 173s 1s/step - loss: 0.0014 - mse: 0.0014 - mae: 0.0214
Epoch 6/180
136/136 [==============================] - 172s 1s/step - loss: 0.0014 - mse: 0.0014 - mae: 0.0206
Epoch 7/180
136/136 [==============================] - 171s 1s/step - loss: 0.0013 - mse: 0.0013 - mae: 0.0202
Epoch 8/180
136/136 [==============================] - 173s 1s/step - loss: 0.0013 - mse: 0.0013 - mae: 0.0200
Epoch 9/180
136/136 [==============================] - 197s 1s/step - loss: 0.0013 - mse: 0.0013 - mae: 0.0196
E

136/136 [==============================] - 187s 1s/step - loss: 7.6563e-04 - mse: 7.6563e-04 - mae: 0.0154
Epoch 72/180
136/136 [==============================] - 185s 1s/step - loss: 0.0012 - mse: 0.0012 - mae: 0.0165
Epoch 73/180
136/136 [==============================] - 187s 1s/step - loss: 8.8067e-04 - mse: 8.8067e-04 - mae: 0.0158
Epoch 74/180
136/136 [==============================] - 195s 1s/step - loss: 8.1008e-04 - mse: 8.1008e-04 - mae: 0.0156
Epoch 75/180
136/136 [==============================] - 184s 1s/step - loss: 8.9149e-04 - mse: 8.9149e-04 - mae: 0.0159
Epoch 76/180
136/136 [==============================] - 185s 1s/step - loss: 7.1413e-04 - mse: 7.1413e-04 - mae: 0.0154
Epoch 77/180
136/136 [==============================] - 186s 1s/step - loss: 6.6726e-04 - mse: 6.6726e-04 - mae: 0.0152
Epoch 78/180
136/136 [==============================] - 182s 1s/step - loss: 9.0732e-04 - mse: 9.0732e-04 - mae: 0.0157
Epoch 79/180
136/136 [==============================] - 175s 

136/136 [==============================] - 175s 1s/step - loss: 5.9145e-04 - mse: 5.9145e-04 - mae: 0.0145
Epoch 140/180
136/136 [==============================] - 177s 1s/step - loss: 5.4638e-04 - mse: 5.4638e-04 - mae: 0.0142
Epoch 141/180
136/136 [==============================] - 175s 1s/step - loss: 6.1076e-04 - mse: 6.1076e-04 - mae: 0.0142
Epoch 142/180
136/136 [==============================] - 173s 1s/step - loss: 5.7466e-04 - mse: 5.7466e-04 - mae: 0.0143
Epoch 143/180
136/136 [==============================] - 177s 1s/step - loss: 5.4380e-04 - mse: 5.4380e-04 - mae: 0.0140
Epoch 144/180
136/136 [==============================] - 178s 1s/step - loss: 5.2742e-04 - mse: 5.2742e-04 - mae: 0.0140
Epoch 145/180
136/136 [==============================] - 177s 1s/step - loss: 5.5224e-04 - mse: 5.5224e-04 - mae: 0.0142
Epoch 146/180
136/136 [==============================] - 174s 1s/step - loss: 7.0930e-04 - mse: 7.0930e-04 - mae: 0.0147
Epoch 147/180
136/136 [=======================

C:\Users\maria\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\maria\anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


Epoch 1/180
113/113 [==============================] - 148s 1s/step - loss: 0.1420 - mse: 0.1420 - mae: 0.2704
Epoch 2/180
113/113 [==============================] - 150s 1s/step - loss: 0.0153 - mse: 0.0153 - mae: 0.0952
Epoch 3/180
113/113 [==============================] - 149s 1s/step - loss: 0.0028 - mse: 0.0028 - mae: 0.0405
Epoch 4/180
113/113 [==============================] - 155s 1s/step - loss: 0.0010 - mse: 0.0010 - mae: 0.0230
Epoch 5/180
113/113 [==============================] - 150s 1s/step - loss: 7.8838e-04 - mse: 7.8838e-04 - mae: 0.0188
Epoch 6/180
113/113 [==============================] - 150s 1s/step - loss: 7.1838e-04 - mse: 7.1838e-04 - mae: 0.0175
Epoch 7/180
113/113 [==============================] - 151s 1s/step - loss: 6.7747e-04 - mse: 6.7747e-04 - mae: 0.0168
Epoch 8/180
113/113 [==============================] - 153s 1s/step - loss: 6.4622e-04 - mse: 6.4622e-04 - mae: 0.0163
Epoch 9/180
113/113 [==============================] - 154s 1s/step - loss: 6.27

113/113 [==============================] - 153s 1s/step - loss: 4.8090e-04 - mse: 4.8090e-04 - mae: 0.0133
Epoch 70/180
113/113 [==============================] - 155s 1s/step - loss: 4.8019e-04 - mse: 4.8019e-04 - mae: 0.0132
Epoch 71/180
113/113 [==============================] - 154s 1s/step - loss: 4.7339e-04 - mse: 4.7339e-04 - mae: 0.0132
Epoch 72/180
113/113 [==============================] - 153s 1s/step - loss: 4.7533e-04 - mse: 4.7533e-04 - mae: 0.0133
Epoch 73/180
113/113 [==============================] - 154s 1s/step - loss: 4.7207e-04 - mse: 4.7207e-04 - mae: 0.0132
Epoch 74/180
113/113 [==============================] - 158s 1s/step - loss: 4.6911e-04 - mse: 4.6911e-04 - mae: 0.0131
Epoch 75/180
113/113 [==============================] - 154s 1s/step - loss: 4.7680e-04 - mse: 4.7680e-04 - mae: 0.0133
Epoch 76/180
113/113 [==============================] - 155s 1s/step - loss: 4.7290e-04 - mse: 4.7290e-04 - mae: 0.0132
Epoch 77/180
113/113 [==============================]

113/113 [==============================] - 161s 1s/step - loss: 3.9379e-04 - mse: 3.9379e-04 - mae: 0.0121
Epoch 138/180
113/113 [==============================] - 161s 1s/step - loss: 3.9983e-04 - mse: 3.9983e-04 - mae: 0.0123
Epoch 139/180
113/113 [==============================] - 159s 1s/step - loss: 3.9568e-04 - mse: 3.9568e-04 - mae: 0.0122
Epoch 140/180
113/113 [==============================] - 161s 1s/step - loss: 3.8350e-04 - mse: 3.8350e-04 - mae: 0.0120
Epoch 141/180
113/113 [==============================] - 159s 1s/step - loss: 3.8688e-04 - mse: 3.8688e-04 - mae: 0.0122
Epoch 142/180
113/113 [==============================] - 162s 1s/step - loss: 3.9538e-04 - mse: 3.9538e-04 - mae: 0.0122
Epoch 143/180
113/113 [==============================] - 156s 1s/step - loss: 3.8740e-04 - mse: 3.8740e-04 - mae: 0.0123
Epoch 144/180
113/113 [==============================] - 157s 1s/step - loss: 3.9664e-04 - mse: 3.9664e-04 - mae: 0.0122
Epoch 145/180
113/113 [=======================

C:\Users\maria\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\maria\anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


Epoch 1/180
90/90 [==============================] - 158s 2s/step - loss: 0.1700 - mse: 0.1700 - mae: 0.3021
Epoch 2/180
90/90 [==============================] - 160s 2s/step - loss: 0.0300 - mse: 0.0300 - mae: 0.1348
Epoch 3/180
90/90 [==============================] - 167s 2s/step - loss: 0.0076 - mse: 0.0076 - mae: 0.0678
Epoch 4/180
90/90 [==============================] - 161s 2s/step - loss: 0.0026 - mse: 0.0026 - mae: 0.0384
Epoch 5/180
90/90 [==============================] - 162s 2s/step - loss: 0.0013 - mse: 0.0013 - mae: 0.0256
Epoch 6/180
90/90 [==============================] - 167s 2s/step - loss: 9.1526e-04 - mse: 9.1526e-04 - mae: 0.0206
Epoch 7/180
90/90 [==============================] - 166s 2s/step - loss: 8.2215e-04 - mse: 8.2215e-04 - mae: 0.0190
Epoch 8/180
90/90 [==============================] - 168s 2s/step - loss: 7.6234e-04 - mse: 7.6234e-04 - mae: 0.0180
Epoch 9/180
90/90 [==============================] - 163s 2s/step - loss: 7.2613e-04 - mse: 7.2613e-04 -

In [ ]:
results = pd.DataFrame({                        
                        'rmse_general': rmse_gen, 
                 
                        'mae_general': mae_gen,
                        
                        'rmse_spike': rmse_spi,
                 
                        'mae_spike': mae_spi,
                        
                        'rmse_normal': rmse_nor,
                    
                        'mae_normal': mae_nor,
    
                        'time': time_count})

#y_pred = pd.DataFrame({'dates': date,
#                       'Predicitons': y_pred_list})

#y_pred.to_csv('Pedictions_LSTM_5_prediction_window.csv')
results.to_csv('Results_LSTM_Prediction_window.csv')
results

In [ ]:
def highlight_min(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.min()
    return ['background-color: yellow' if v else '' for v in is_max]

results.style.apply(highlight_min)

In [ ]:
results

In [ ]:
%matplotlib notebook

fontsize = 13

date_labels =   ['12', 
                 '10', 
                 '8',
                 '6', 
                 '4', 
                 '2']

plt.figure(figsize=(10,3.5))
plt.minorticks_on()
plt.grid(which='major', linestyle='-', linewidth='0.5')
plt.grid(which='minor', linestyle=':', linewidth='0.5')
plt.title('ANN: Averaged RMSE for different prediction windows', fontsize = fontsize + 2)
plt.plot([0,1,2,3,4,5], rmse_gen, label = 'On the whole test set')
plt.plot([0,1,2,3,4,5], rmse_spi, label = 'Spike regions')
plt.plot([0,1,2,3,4,5], rmse_nor, label = 'Non-spike regions')
plt.legend(loc = 'upper right', fontsize = fontsize - 1)
plt.ylabel('RMSE (£/MWh)', fontsize = fontsize)
plt.xlabel('Predictive window (in months)', fontsize = fontsize)
plt.xticks([0,1,2,3,4,5], list(np.arange(2, 14, 2))[::-1], fontsize = fontsize)
plt.yticks(fontsize = fontsize)
plt.xlim(0, 5)
plt.tight_layout()
plt.savefig('RMSE_predictive_window.png')

plt.figure(figsize=(10,3.5))
plt.minorticks_on()
plt.grid(which='major', linestyle='-', linewidth='0.5')
plt.grid(which='minor', linestyle=':', linewidth='0.5')
plt.title('Random Forest: Averaged MAE for different prediction windows', fontsize = fontsize + 2)
plt.plot([0,1,2,3,4,5], rmse_gen, label = 'On the whole test set')
plt.plot([0,1,2,3,4,5], rmse_spi, label = 'Spike regions')
plt.plot([0,1,2,3,4,5], rmse_nor, label = 'Non-spike regions')
plt.legend(loc = 'upper right', fontsize = fontsize - 1)
plt.ylabel('MAE (£/MWh)', fontsize = fontsize)
plt.xlabel('Predictive window (in months)', fontsize = fontsize)
plt.xticks([0,1,2,3,4,5], list(np.arange(2, 14, 2))[::-1], fontsize = fontsize)
plt.yticks(fontsize = fontsize)
plt.xlim(0, 5)
plt.tight_layout()
plt.savefig('MAE_predictive_window.png')

In [ ]:
%matplotlib notebook

plt.plot(y_pred)
plt.plot(y_test)